# Vector Database Basics

Vector databases help us store, manage, and query the embeddings we created for generative AI, recommenders, and search engines.

Across many of the common use cases, users often find that they need to manage more than just vectors.
To make it easier for practitioners, vector databases should store and manage all of the data they need:
- embedding vectors
- categorical metadata
- numerical metadata
- timeseries metadata
- text / pdf / images / video / point clouds

And support a wide range of query workloads:
- Vector search (may require ANN-index)
- Keyword search (requires full text search index)
- SQL (for filtering)

For this exercise we'll use LanceDB since it's open source and easy to setup

In [ ]:
# pip install -U --quiet lancedb pandas pydantic

## Creating tables and adding data

Let's create a LanceDB table called `cats_and_dogs` under the local database directory `~/.lancedb`.
This table should have 4 fields:
- the embedding vector
- a string field indicating the species (either "cat" or "dog")
- the breed
- average weight in pounds

We're going to use pydantic to make this easier. First let's create a pydantic model with those fields

In [ ]:
from lancedb.pydantic import vector, LanceModel

class CatsAndDogs(LanceModel):
    vector: vector(2)
    species: str
    breed: str
    weight: float

Now connect to a local db at ~/.lancedb and create an empty LanceDB table called "cats_and_dogs"

In [ ]:
import lancedb

db = lancedb.connect("~/.lancedb")
table_name = "cats_and_dogs"
db.drop_table(table_name, ignore_missing=True)
table = db.create_table(table_name, schema=CatsAndDogs)

Let's add some data

First some cats

In [ ]:
data = [
    CatsAndDogs(
        vector=[1., 0.],
        species="cat",
        breed="shorthair",
        weight=12.,
    ),
    CatsAndDogs(
        vector=[-1., 0.],
        species="cat",
        breed="himalayan",
        weight=9.5,
    ),
]

Now call the `LanceTable.add` API to insert these two records into the table

In [ ]:
table.add([dict(d) for d in data])

Let's preview the data

In [ ]:
table.head().to_pandas()

,vector,species,breed,weight
0,"[1.0, 0.0]",cat,shorthair,12.0
1,"[-1.0, 0.0]",cat,himalayan,9.5


Now let's add some dogs

In [ ]:
data = [
    CatsAndDogs(
        vector=[0., 10.],
        species="dog",
        breed="samoyed",
        weight=47.5,
    ),
    CatsAndDogs(
        vector=[0, -1.],
        species="dog",
        breed="corgi",
        weight=26.,
    )
]

In [ ]:
table.add([dict(d) for d in data])

In [ ]:
table.head().to_pandas()

,vector,species,breed,weight
0,"[1.0, 0.0]",cat,shorthair,12.0
1,"[-1.0, 0.0]",cat,himalayan,9.5
2,"[0.0, 10.0]",dog,samoyed,47.5
3,"[0.0, -1.0]",dog,corgi,26.0


## Querying tables

Vector databases allow us to retrieve data for generative AI applications. Let's see how that's done.

Let's say we have a new animal that has embedding [10.5, 10.], what would we expect the most similar animal will be?
Can we use the table we created above to answer the question?

We'll need to use the `search` API for LanceTable and `limit` / `to_df` APIs. We can refer to [LanceDB documentation](https://lancedb.github.io/lancedb/basic/#how-to-search-for-approximate-nearest-neighbors).

We if get "samoyed" then we are right!. Here we pass the vector into `search` and make a chained call to `limit` with 1 as the param. Then we'll need to call `to_df` to execute the query and convert the results to a pandas dataframe. In addition to the data columns, we'll also see a "score" column, which contains the distance score between the query vector and the returned vector. In this case, the score is the square of the Euclidean distance.

In [ ]:
table.search([10.5, 10.,]).limit(1).to_df()

,vector,species,breed,weight,_distance
0,"[0.0, 10.0]",dog,samoyed,47.5,110.25


Now what if we use cosine distance instead? Would we expect that we get the same answer?

We can add a call to `metric` in the call chain

Cosine distance is the angle between vectors. So while the query vector [10.5, 10.] is closer in Euclidean distance to Samoyed, the angle is slightly closer to shorthair cats.

In [ ]:
table.search([10.5, 10.]).metric("cosine").limit(1).to_df()

,vector,species,breed,weight,_distance
0,"[1.0, 0.0]",cat,shorthair,12.0,0.275862


## Filtering tables

In practice, we often need to specify more than just a search vector for good quality retrieval. Oftentimes we need to filter the metadata as well.

We write code to retrieve two most similar examples to the embedding [10.5, 10.] but only show the results that is a cat.
In LanceDB, for additional filtering, we can add a call to `where` in the call chain and pass in a SQL-like filter string.

In [ ]:
table.search([10.5, 10.,]).limit(2).where("species='cat'").to_df()

,vector,species,breed,weight,_distance
0,"[1.0, 0.0]",cat,shorthair,12.0,190.25


## Creating ANN indices

For larger tables (e.g., >1M rows), searching through all of the vectors becomes quite slow. Here is where the Approximate Nearest Neighbor (ANN) index comes into play. While there are many different ANN indexing algorithms, they all have the same purpose - to drastically limit the search space as much as possible while losing as little accuracy as possible

For this problem we will create an ANN index on a LanceDB table and see how that impacts performance

### First let's create some data

Given the constraints of the workspace, we'll complete this exercise by creating 100,000 vectors with 16D in a new table. Here the embedding values don't matter, so we simply generate random embeddings as a 2D numpy array. We then use the `vec_to_table` function to convert that in to an Arrow table, which can then be added to the table.

In [ ]:
from lance.vector import vec_to_table
import numpy as np

mat = np.random.randn(100_000, 16)
table_name = "exercise3_ann"
db.drop_table(table_name, ignore_missing=True)
table = db.create_table(table_name, vec_to_table(mat))

### Let's establish a baseline without an index

Before we create the index, let's make sure know what we need to compare against.

We'll generate a random query vector and record it's value in the `query` variable so we can use the same query vector with and without the ANN index.

In [ ]:
query = np.random.randn(16)
table.search(query).limit(10).to_df()

,vector,_distance
0,"[-0.1767028, -0.5229317, -1.4283801, 0.0392339...",5.347155
1,"[0.09929892, -0.073682174, -1.1638305, -1.0271...",5.735785
2,"[0.81666917, -0.37490085, -1.5122684, -1.78993...",5.888507
3,"[-0.6520697, -0.37692794, -2.0548658, 0.051603...",6.202878
4,"[0.43040285, -0.2845944, -1.7629384, 0.1087845...",6.444257
5,"[0.3332895, -0.974127, -1.4170854, -0.75394565...",6.712359
6,"[-0.29099643, -0.13922098, -1.6444877, -0.7764...",6.763872
7,"[-0.5022906, -0.5896424, -0.7970898, 0.1395855...",6.834733
8,"[-0.09440635, -0.22170915, -0.93469584, -0.021...",6.972491
9,"[0.16971456, -0.62072587, -0.53135717, -0.6371...",7.018113


Code to compute the average latency of this query

There are several possible solutions. Given that we're in a notebook environment, the easiest is probably using the %timeit magic function to run the command a bunch of times and compute the average

In [ ]:
%timeit table.search(np.random.randn(16)).limit(10).to_arrow();

11.2 ms ± 352 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Now let's create an index

There are many possible index types ranging from hash based to tree based to partition based to graph based.
For this task, we'll create an IVFPQ index (partition-based index with product quantization compression) using LanceDB.

Please create an IVFPQ index on the LanceDB table such that each partition is 4000 rows and each PQ subvector is 8D.

**HINT**
1. Total vectors / number of partitions = number of vectors in each partition
2. Total dimensions / number of subvectors = number of dimensions in each subvector

In [ ]:
table.create_index(num_partitions=16, num_sub_vectors=8)

Sample 4096 out of 65536 to train kmeans of 16 dim, 16 clusters


Now let's search through the data again. Notice how the answers now appear different.
This is because an ANN index is always a tradeoff between latency and accuracy.

In [ ]:
table.search(query).limit(10).to_df()

,vector,_distance
0,"[-0.1767028, -0.5229317, -1.4283801, 0.0392339...",5.024755
1,"[0.81666917, -0.37490085, -1.5122684, -1.78993...",5.092604
2,"[0.09929892, -0.073682174, -1.1638305, -1.0271...",6.207117
3,"[-0.29099643, -0.13922098, -1.6444877, -0.7764...",6.269482
4,"[-0.6520697, -0.37692794, -2.0548658, 0.051603...",6.611748
5,"[0.43040285, -0.2845944, -1.7629384, 0.1087845...",6.706854
6,"[0.3332895, -0.974127, -1.4170854, -0.75394565...",6.764954
7,"[-0.09440635, -0.22170915, -0.93469584, -0.021...",7.031804
8,"[0.16971456, -0.62072587, -0.53135717, -0.6371...",7.062588
9,"[1.1067225, -0.52280897, -1.3419323, -1.009257...",7.151832


Code to compute the average latency for querying the same table using the ANN index.
The index is implementation detail, so it should just be running the same code as above. We should see almost an order of magnitude speed-up. On larger datasets, this performance difference should be even more pronounced.

In [ ]:
%timeit table.search(np.random.randn(16)).limit(10).to_arrow();

9.18 ms ± 299 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Deleting rows

Like with other kinds of databases, we should be able to remove rows from the table.
Let's go back to our tables of cats and dogs

In [ ]:
table = db["cats_and_dogs"]

In [ ]:
len(table)

4

Can we use the `delete` API to remove all of the cats from the table?

Use a SQL like filter string to specify which rows to delete from the table

In [ ]:
table.delete("species='cat'")

In [ ]:
len(table)

2

*italicized text*## What if we messed up?

Errors is a common occurrence in AI. What's hard about errors in vector search is that oftentimes a bad vector doesn't cause a crash but just creates non-sensical answers. So to be able to rollback the state of the database is very important for debugging and reproducibility

So far we've accumulated 4 actions on the table:
1. creation of the table
2. added cats
3. added dogs
4. deleted cats

What if we realized that we should have deleted the dogs instead of the cats?

Here we can see the 4 versions that correspond to the 4 actions we've done

In [ ]:
table.list_versions()

[{'version': 1,
  'timestamp': datetime.datetime(2025, 7, 29, 2, 3, 36, 188040),
  'metadata': {}},
 {'version': 2,
  'timestamp': datetime.datetime(2025, 7, 29, 2, 3, 39, 628141),
  'metadata': {}},
 {'version': 3,
  'timestamp': datetime.datetime(2025, 7, 29, 2, 3, 41, 957229),
  'metadata': {}},
 {'version': 4,
  'timestamp': datetime.datetime(2025, 7, 29, 2, 7, 30, 768484),
  'metadata': {}}]

Code to restore the version still containing the whole dataset

In [ ]:
table = db["cats_and_dogs"]

In [ ]:
len(table)

2

In [ ]:
table.restore(3)

In [ ]:
table.delete("species='dog'")

In [ ]:
table.list_versions()

[{'version': 1,
  'timestamp': datetime.datetime(2025, 7, 29, 2, 3, 36, 188040),
  'metadata': {}},
 {'version': 2,
  'timestamp': datetime.datetime(2025, 7, 29, 2, 3, 39, 628141),
  'metadata': {}},
 {'version': 3,
  'timestamp': datetime.datetime(2025, 7, 29, 2, 3, 41, 957229),
  'metadata': {}},
 {'version': 4,
  'timestamp': datetime.datetime(2025, 7, 29, 2, 7, 30, 768484),
  'metadata': {}},
 {'version': 5,
  'timestamp': datetime.datetime(2025, 7, 29, 2, 7, 30, 813583),
  'metadata': {}},
 {'version': 6,
  'timestamp': datetime.datetime(2025, 7, 29, 2, 7, 30, 826009),
  'metadata': {}}]

In [ ]:
table.to_pandas()

,vector,species,breed,weight
0,"[1.0, 0.0]",cat,shorthair,12.0
1,"[-1.0, 0.0]",cat,himalayan,9.5


## Dropping a table

We can also choose to drop a table, which also completely removes the data.
Note that this operation is not reversible.

In [ ]:
"cats_and_dogs" in db

True

Write code to irrevocably remove the table "cats_and_dogs" from the database

In [ ]:
db.drop_table("cats_and_dogs")

How would we verify that the table has indeed been deleted?

In [ ]:
table.name in db

False